<a href="https://colab.research.google.com/github/alfreddey/smart-food-analyzer-/blob/main/training-script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive

In [7]:
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [9]:
data_dir = "/content/drive/MyDrive/tfds-food-101"

In [10]:
(train_ds, val_ds, test_ds), ds_info = tfds.load(
    'food101',
    split=['train', 'validation', "validation[50%:]"],
    as_supervised=True,
    shuffle_files=True,
    data_dir=data_dir,
    with_info=True
)

In [11]:
print(f"The number of training samples {train_ds.cardinality()}")
print(f"The number of validation samples {val_ds.cardinality()}")
print(f"The number of test samples {test_ds.cardinality()}")

The number of training samples 75750
The number of validation samples 25250
The number of test samples 12625


In [12]:
img_size = (224, 224, 3)
batch_size = 32
epochs = 20
num_class = 101

In [13]:
resize = tf.keras.layers.Resizing(224, 224)

In [14]:
#resize datasets
train_ds = train_ds.map(lambda x, y: (resize(x), y))
val_ds = val_ds.map(lambda x, y: (resize(x), y))
test_ds = test_ds.map(lambda x, y: (resize(x), y))

In [15]:
train_ds = train_ds.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [16]:
base_model = tf.keras.applications.MobileNetV2(
    weights="imagenet",
    input_shape=img_size,
    include_top=False,
    classifier_activation='softmax'
)

base_model.trainable = False


inputs = tf.keras.layers.Input(shape=img_size)

scale_layer = tf.keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
x = scale_layer(inputs)

x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(num_class, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

model.summary(show_trainable=True)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ input_layer_1 (InputLayer)  │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ rescaling (Rescaling)       │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ mobilenetv2_1.00_224        │ (None, 7, 7, 1280)    │  2,257,984 │   N   │
│ (Functional)                │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ global_average_pooling2d    │ (None, 1280)          │          0 │   -   │
│ (GlobalAveragePooling2D)    │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dropout (Dropout)           │ (None, 1280)          │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense (Dense)               │ (None, 101)           │    129,381 │   Y   │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 2,387,365 (9.11 MB)

 Trainable params: 129,381 (505.39 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [17]:
earlystop_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

In [23]:
def plot_history(hist, title):
    plt.figure()
    plt.plot(hist.history['accuracy'], label='train_acc')
    plt.plot(hist.history['val_accuracy'], label='val_acc')
    plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [18]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

print("Fitting the top layer of the model")
history = model.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=[earlystop_cb])

Fitting the top layer of the model
Epoch 1/20
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 341s 105ms/step - accuracy: 0.3851 - loss: 2.6049 - val_accuracy: 0.6065 - val_loss: 1.4931
Epoch 2/20
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 214s 89ms/step - accuracy: 0.5682 - loss: 1.6978 - val_accuracy: 0.6125 - val_loss: 1.4711
Epoch 3/20
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 290s 101ms/step - accuracy: 0.5887 - loss: 1.5892 - val_accuracy: 0.6172 - val_loss: 1.4654
Epoch 4/20
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 213s 89ms/step - accuracy: 0.6067 - loss: 1.5209 - val_accuracy: 0.6154 - val_loss: 1.4585
Epoch 5/20
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 296s 103ms/step - accuracy: 0.6139 - loss: 1.4781 - val_accuracy: 0.6197 - val_loss: 1.4475
Epoch 6/20
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 259s 102ms/step - accuracy: 0.6166 - loss: 1.4604 - val_accuracy: 0.6121 - val_loss: 1.4957
Epoch 7/20
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 269s 105ms/step - accuracy: 0.6214 - loss: 1.4402 - val_accuracy: 0.6187 - val_loss: 1.4823
Epoch 8/20
2368/2368 ━━━━━━━━

In [29]:
plot_history(history, 'Initial Training Accuracy')

AttributeError: 'NoneType' object has no attribute 'history'

<Figure size 640x480 with 0 Axes>

In [19]:
model.evaluate(test_ds)

395/395 ━━━━━━━━━━━━━━━━━━━━ 27s 68ms/step - accuracy: 0.6182 - loss: 1.4376


[1.4507168531417847, 0.614653468132019]

In [20]:
#fine-tuning
base_model.trainable = True;

In [21]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=['accuracy']
)

In [36]:
history2 = model.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=[earlystop_cb])

Epoch 1/20
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 348s 146ms/step - accuracy: 0.8738 - loss: 0.4446 - val_accuracy: 0.7438 - val_loss: 0.9690
Epoch 2/20
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 350s 132ms/step - accuracy: 0.8840 - loss: 0.4073 - val_accuracy: 0.7437 - val_loss: 0.9770
Epoch 3/20
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 351s 147ms/step - accuracy: 0.8897 - loss: 0.3809 - val_accuracy: 0.7429 - val_loss: 0.9794
Epoch 4/20
   3/2368 ━━━━━━━━━━━━━━━━━━━━ 4:43 120ms/step - accuracy: 0.8576 - loss: 0.3613

KeyboardInterrupt: 

In [24]:
plot_history(history2, 'Fine-tuning Accuracy')

NameError: name 'history2' is not defined

In [33]:
print('Evaluation After fine-tuning')
model.evaluate(test_ds)

Evaluation After fine-tuning
395/395 ━━━━━━━━━━━━━━━━━━━━ 31s 78ms/step - accuracy: 0.7380 - loss: 0.9746


[0.9702285528182983, 0.7394059300422668]

In [34]:
tf.saved_model.save(model, 'model')

In [27]:
!pip install tensorflowjs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 3.35.0 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.
xarray 2025.7.1 requires packaging>=24.1, but you have packaging 23.2 which is incompatible.
db-dtypes 1.4.3 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.


In [35]:
!tensorflowjs_converter \
    --input_format=tf_saved_model \
    model/ \
    web_model

2025-07-27 01:22:52.936066: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753579373.243764   38750 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753579373.325001   38750 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
🌲 Try https://ydf.readthedocs.io, the successor of TensorFlow Decision Forests with more features and faster training!
2025-07-27 01:23:05.536578: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0000 00:00:1753579385.536736   38750 gpu_device.cc:2022] Created device /job:localhost/replica:0/task